In [28]:
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed
from keras.layers import Dense, Input
from keras.layers import GlobalMaxPooling1D, Bidirectional
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
import pickle

In [2]:
bird_audio_with_label = pickle.load( open( "bird.txt", "rb" ) )
X_bird = bird_audio_with_label["bird"]
Y_bird = np.array([[1,0]] * X_bird.shape[0])
print X_bird.shape
print Y_bird.shape

bed_audio_with_label = pickle.load( open( "bed.txt", "rb" ) )
X_bed = bed_audio_with_label["bed"]
Y_bed = np.array([[0,1]] * X_bed.shape[0])
print X_bed.shape
print Y_bed.shape

X = np.vstack((X_bird, X_bed))
Y = np.vstack((Y_bird, Y_bed))
print X.shape
print Y.shape

(1731, 20, 100)
(1731, 2)
(1713, 20, 100)
(1713, 2)
(3444, 20, 100)
(3444, 2)


In [3]:
X_ = np.array([[[1,2,3,4,5,6],[7,8,9,10,11,12]]])
Y_ = np.array([[1,0]])
#Y = np.array([[[1],[2]]])
print X_.shape
print Y_.shape

(1, 2, 6)
(1, 2)


In [18]:
inp = Input(shape=(20,100 ))
x = (LSTM(100, return_sequences=True))(inp)
x = (LSTM(100, return_sequences=True))(x)
x = (LSTM(100, return_sequences=True))(x)
x = GlobalMaxPooling1D()(x)
x = Dropout(0.4)(x)

#x = Dense(20, activation = "sigmoid")(x)
#x = Dense(5, activation = "sigmoid")(x)

x = Dense(2, activation="softmax")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 20, 100)           0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 20, 100)           80400     
_________________________________________________________________
lstm_12 (LSTM)               (None, 20, 100)           80400     
_________________________________________________________________
lstm_13 (LSTM)               (None, 20, 100)           80400     
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 100)               0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 202       
Total para

In [26]:
file_path="RNN_best_val_loss.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early = EarlyStopping(monitor="val_loss", mode="min", patience=20)
callbacks_list = [checkpoint, early] #early
model.fit(X, Y, validation_split=0.1, callbacks=callbacks_list, epochs=10)

Train on 3099 samples, validate on 345 samples
Epoch 1/10
3099/3099 [==============================] - 10s - loss: 0.1702 - acc: 0.9316 - val_loss: 1.0488 - val_acc: 0.6551
Epoch 2/10
3099/3099 [==============================] - 10s - loss: 0.1471 - acc: 0.9419 - val_loss: 0.5395 - val_acc: 0.8058
Epoch 3/10
3099/3099 [==============================] - 10s - loss: 0.1334 - acc: 0.9497 - val_loss: 0.4990 - val_acc: 0.8290
Epoch 4/10
3099/3099 [==============================] - 10s - loss: 0.1238 - acc: 0.9555 - val_loss: 0.7827 - val_acc: 0.7449
Epoch 5/10
3099/3099 [==============================] - 10s - loss: 0.1389 - acc: 0.9451 - val_loss: 1.1052 - val_acc: 0.6754
Epoch 6/10
3099/3099 [==============================] - 10s - loss: 0.1082 - acc: 0.9564 - val_loss: 0.7727 - val_acc: 0.7797
Epoch 7/10
3099/3099 [==============================] - 10s - loss: 0.1002 - acc: 0.9664 - val_loss: 0.8098 - val_acc: 0.7362
Epoch 8/10
3099/3099 [==============================] - 10s - loss: 0.0

In [30]:
model = load_model(file_path)

In [35]:
a= model.predict(X[2:3])

In [36]:
a

array([[ 0.99343562,  0.00656441]], dtype=float32)

In [50]:
X[3443:3444].shape

(1, 20, 100)